In [20]:
import os
import time
from datetime import datetime
import pandas as pd
from atproto import Client
import keyring

# Constants
ROOT_HANDLE = "elaval.bsky.social"
STATE_FILES = {
    "log": "followers_log.parquet",
    "master": "master.parquet",
}
RATE_LIMIT_DELAY = 0.2  # Delay between API calls to respect rate limits
EXEMPTIONS = {"ap.brid.gy"}

def get_password():
    """Retrieve the password based on the environment."""
    # Check if running in a GitHub Actions environment
    github_password = os.getenv("BLUESKY_PASSWORD")
    if github_password:
        print("Using GitHub Actions secret for password.")
        return github_password

    # Otherwise, use keyring for local development
    print("Using keyring for password.")
    password = keyring.get_password("bluesky", ROOT_HANDLE)
    if not password:
        raise ValueError(
            "Password not found in keyring. Please set it using keyring for local use or as a GitHub secret for CI/CD."
        )
    return password

# Helper Functions
def save_parquet(file_name, data):
    """Save data to a parquet file."""
    data.to_parquet(file_name, index=False)

def load_parquet(file_name, columns=None):
    """Load data from a parquet file, initializing if missing."""
    if os.path.exists(file_name):
        return pd.read_parquet(file_name)
    # Initialize with the correct schema if the file doesn't exist
    if columns:
        return pd.DataFrame(columns=columns)
    return pd.DataFrame()

# BlueSky Functions
def fetch_profile(client, handle):
    """Fetch a user profile from the BlueSky API."""
    try:
        print(f"Fetching profile for {handle}...")
        profile = client.get_profile(actor=handle)
        record = {
            "handle": handle,
            "displayName": profile.display_name or "",
            "description": profile.description or "",
            "followers_count": profile.followers_count or 0,
            "created_at": profile.created_at,
            "avatar_url": profile.avatar or "",
            "timestamp": datetime.now(),
        }
        print(f"Fetched profile: {record}")
        return record
    except Exception as e:
        print(f"Error fetching profile for {handle}: {e}")
        return None

def fetch_followers(client, handle):
    """Fetch the list of followers for a given user."""
    followers = []
    cursor = None
    try:
        print(f"Fetching followers for {handle}...")
        while True:
            response = client.get_followers(actor=handle, limit=50, cursor=cursor)
            followers.extend([f.handle for f in response.followers])
            if not response.cursor:
                break
            cursor = response.cursor
            time.sleep(RATE_LIMIT_DELAY)
    except Exception as e:
        print(f"Error fetching followers for {handle}: {e}")
    return followers

# Data Update Functions
def update_master_file(profile, master_data):
    """Update the master file with the latest profile information."""
    new_row = pd.DataFrame([{
        "handle": profile["handle"],
        "displayName": profile["displayName"],
        "description": profile["description"],
        "followers_count": profile["followers_count"],
        "created_at": profile.get("created_at", ""),
        "avatar_url": profile["avatar_url"],
        "timestamp": profile["timestamp"],
    }])
    master_data = master_data[master_data["handle"] != profile["handle"]]
    master_data = pd.concat([master_data, new_row], ignore_index=True)
    return master_data

def update_log_file(profile, log_data):
    """Update the log file with the latest follower count."""
    new_row = pd.DataFrame([{
        "handle": profile["handle"],
        "followers_count": profile["followers_count"],
        "timestamp": profile["timestamp"],
    }])
    log_data = pd.concat([log_data, new_row], ignore_index=True)
    return log_data

# Main Processing
def process_followers(client, root_handle, master_file, log_file):
    """Process followers for the root handle."""
    # Initialize column structure
    master_columns = [
        "handle", "displayName", "description", "followers_count", "created_at", "timestamp"
    ]
    log_columns = ["handle", "followers_count", "timestamp"]

    # Load existing data
    master_data = load_parquet(master_file, columns=master_columns)
    log_data = load_parquet(log_file, columns=log_columns)

    # Process the root user's profile
    root_profile = fetch_profile(client, root_handle)
    if root_profile:
        print("Updating master and log for root user...")
        master_data = update_master_file(root_profile, master_data)
        log_data = update_log_file(root_profile, log_data)

    # Fetch and process followers
    print("Requesting followers...")
    followers = fetch_followers(client, root_handle)
    print(f"Got {len(followers)} followers.")
    for i, follower in enumerate(followers, start=1):
        if follower in EXEMPTIONS:
            print(f"Skipping {follower} due to exemption.")
            continue

        print(f"Processing follower {follower} ({i}/{len(followers)})...")
        profile = fetch_profile(client, follower)
        if profile:
            try:
                master_data = update_master_file(profile, master_data)
                log_data = update_log_file(profile, log_data)
            except Exception as e:
                print(f"Error updating master/log for {follower}: {e}")

        # Save state periodically
        if i % 10 == 0 or i == len(followers):
            print(f"Saving state after processing {i} followers...")
            save_parquet(master_file, master_data)
            save_parquet(log_file, log_data)

    # Final save
    print("Final save of master and log data.")
    save_parquet(master_file, master_data)
    save_parquet(log_file, log_data)

# Main Function
def main():
    elmtest = os.getenv("ELMTEST")
    if elmtest:
        print("Got elmtest.")
        print(elmtest)
    
    password = get_password()
    client = Client()
    client.login(ROOT_HANDLE, password)
    profile = client.get_profile('elaval.bsky.social')
    print(profile)

    # Process followers and update data files
    try:
        print("Main try")
        process_followers(client, ROOT_HANDLE, STATE_FILES["master"], STATE_FILES["log"])
    except KeyboardInterrupt:
        print("Interrupted by user. Saving state...")
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()


Using keyring for password.
did='did:plc:zow572l72hxangbdue3usuw3' handle='elaval.bsky.social' associated=ProfileAssociated(chat=None, feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated') avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:zow572l72hxangbdue3usuw3/bafkreicosxnyjfcthenwwjah5by5fnpqpkpngekutnc4r2nyfnne7fyqce@jpeg' banner=None created_at='2024-11-16T18:43:08.875Z' description='Me gusta transformar datos en información con sentido. \n\nEn la Universidad estudié Ingeniería y Educación ... pero aprendo día a día en la vida. \n\n❤️ a mi familia' display_name='Ernesto Laval' followers_count=844 follows_count=79 indexed_at='2024-11-20T09:39:57.269Z' joined_via_starter_pack=None labels=[] pinned_post=None posts_count=38 viewer=ViewerState(blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following=None, known_followers=KnownFollowers(count=31, followers=[ProfileViewBasic(did='did:plc:gvboqghesbqlv73klkwdq

Fetched profile: {'handle': 'tutelkan.bsky.social', 'displayName': 'Tutelkan', 'description': 'Creamos software a la medida que optimizan tu negocio, solucionan problemas y mejoran los procesos de tu empresa sin importar su tamaño 💻\u200b🔧\u200b🧑🏻\u200d💻\u200b\u200b📈\u200b👍\u200b  tutelkan.com', 'followers_count': 4, 'created_at': '2024-11-28T19:54:54.846Z', 'avatar_url': 'https://cdn.bsky.app/img/avatar/plain/did:plc:li6dqjrnq3my376wxlo2tgna/bafkreihvgp2qnndwb2cemohwxhbjyoa7lkva7g7n4uflbhu33zg7ryiyh4@jpeg', 'timestamp': datetime.datetime(2024, 12, 4, 20, 35, 32, 533873)}
Processing follower cgarciac.bsky.social (4/844)...
Fetching profile for cgarciac.bsky.social...
Fetched profile: {'handle': 'cgarciac.bsky.social', 'displayName': '', 'description': '', 'followers_count': 3, 'created_at': '2024-12-04T18:38:59.846Z', 'avatar_url': 'https://cdn.bsky.app/img/avatar/plain/did:plc:oe7jf5x36chumljujcnvsndf/bafkreidqobxw5n6xmum64ruxywlagklbd566nebloiqh7m3ayawiwqv7em@jpeg', 'timestamp': date

Fetched profile: {'handle': 'rojasduranpablo.bsky.social', 'displayName': 'Pablo Rojas', 'description': 'Funcionario público de Culturas\nTransitando por La Dormida\nEducación + Artes + Arquitectura', 'followers_count': 99, 'created_at': '2024-11-17T13:01:25.830Z', 'avatar_url': 'https://cdn.bsky.app/img/avatar/plain/did:plc:dcztljh6juxb6k22qln2fqs6/bafkreif2q3omj6eo4qriooyrlj6c4qmwjqr3czczxrd35kriqkycmxk6si@jpeg', 'timestamp': datetime.datetime(2024, 12, 4, 20, 35, 37, 225347)}
Processing follower andrea-cs.bsky.social (20/844)...
Fetching profile for andrea-cs.bsky.social...
Fetched profile: {'handle': 'andrea-cs.bsky.social', 'displayName': 'Andrea', 'description': 'Eterna aprendiz\n📚📽️📻🎧🐈🏖️🏔️', 'followers_count': 305, 'created_at': '2024-11-14T13:55:45.027Z', 'avatar_url': 'https://cdn.bsky.app/img/avatar/plain/did:plc:oz2apzyls6po2vbgwahso7tj/bafkreigkhnsfezj72baplelvwqeslvjidot4moxhrv7pji5lboeia3x3d4@jpeg', 'timestamp': datetime.datetime(2024, 12, 4, 20, 35, 37, 534303)}
Saving s

Fetched profile: {'handle': 'aliuejatta.bsky.social', 'displayName': '', 'description': '', 'followers_count': 472, 'created_at': '2024-11-24T23:28:37.146Z', 'avatar_url': 'https://cdn.bsky.app/img/avatar/plain/did:plc:zeh3kt6tscpwxpsisauknq6f/bafkreidaq2nurcna2izfnnsgdetghzfnm2yga5cz6pftwrrt3kk5zgasru@jpeg', 'timestamp': datetime.datetime(2024, 12, 4, 20, 35, 42, 232992)}
Processing follower isacor.bsky.social (36/844)...
Fetching profile for isacor.bsky.social...
Fetched profile: {'handle': 'isacor.bsky.social', 'displayName': 'Isa', 'description': 'PhD mathematics, associate professor.\n\n(Hago listas musicales:\nhttps://linktr.ee/mariaisabel.cortez?utm_source=linktree_profile_share&ltsid=ed552ba5-e18f-4198-98ca-82a8a0d9f280', 'followers_count': 105, 'created_at': '2024-11-17T22:54:03.684Z', 'avatar_url': 'https://cdn.bsky.app/img/avatar/plain/did:plc:b575attkfduwavvteoe3mplv/bafkreiexvjldx5v3i3zpoedlkboznsavw7rkje3l66zsvearboefa6jxba@jpeg', 'timestamp': datetime.datetime(2024, 12, 

Fetched profile: {'handle': 'ibarraseb.bsky.social', 'displayName': '', 'description': '', 'followers_count': 7, 'created_at': '2024-11-24T21:30:33.346Z', 'avatar_url': 'https://cdn.bsky.app/img/avatar/plain/did:plc:y6bcx7ibe4j7c3l7wgt37fdz/bafkreifuqsgcsd2lrpzdi5zemmdk7iftplwpav6wubhowaq2cfqtac3ize@jpeg', 'timestamp': datetime.datetime(2024, 12, 4, 20, 35, 48, 34187)}
Processing follower franpla.bsky.social (52/844)...
Fetching profile for franpla.bsky.social...
Fetched profile: {'handle': 'franpla.bsky.social', 'displayName': '', 'description': '', 'followers_count': 11, 'created_at': '2024-11-16T21:51:37.727Z', 'avatar_url': 'https://cdn.bsky.app/img/avatar/plain/did:plc:kfr5mjn6tkeo376axwn3vgsi/bafkreid6gijh23bqd5uopgnntcmjo2mcosyoudfql7quwjsvcvd6d2xm3i@jpeg', 'timestamp': datetime.datetime(2024, 12, 4, 20, 35, 48, 333824)}
Processing follower rodriguezluis.bsky.social (53/844)...
Fetching profile for rodriguezluis.bsky.social...
Fetched profile: {'handle': 'rodriguezluis.bsky.soci

...

Fetched profile: {'handle': 'kinesioloco.bsky.social', 'displayName': 'Don Panchisco', 'description': 'Magallanico de corazón, osornino por hábito.\nKinesiologo en APS.\nExistencialista.', 'followers_count': 104, 'created_at': '2024-11-16T23:57:49.050Z', 'avatar_url': 'https://cdn.bsky.app/img/avatar/plain/did:plc:lykraoc6b4vtymjcux776fuk/bafkreicqmlm2pwjs6zl6g5rcuxck64z4ngfsaakwmjbv2awvf4epwlhmzq@jpeg', 'timestamp': datetime.datetime(2024, 12, 4, 20, 39, 37, 292696)}
Processing follower yasnacarolagr.bsky.social (776/844)...
Fetching profile for yasnacarolagr.bsky.social...
Fetched profile: {'handle': 'yasnacarolagr.bsky.social', 'displayName': '', 'description': '', 'followers_count': 38, 'created_at': '2024-11-18T18:03:05.149Z', 'avatar_url': 'https://cdn.bsky.app/img/avatar/plain/did:plc:q6wdi5fr7mh3thgokgavydir/bafkreifn5eddseht3uuxqdijhkiclytzpmdfbblq3btbt4iciz2alpceii@jpeg', 'timestamp': datetime.datetime(2024, 12, 4, 20, 39, 37, 598355)}
Processing follower locaenelcamino.bsky.

Fetched profile: {'handle': 'miguelohi.bsky.social', 'displayName': '', 'description': '', 'followers_count': 29, 'created_at': '2024-11-14T16:29:52.237Z', 'avatar_url': 'https://cdn.bsky.app/img/avatar/plain/did:plc:w7lrvcipyr6ono7cw5gzpsmg/bafkreifguxs7rphq4aq75zpil7kmw6nxfi6anamvicppp72y7lz5xxxhgi@jpeg', 'timestamp': datetime.datetime(2024, 12, 4, 20, 39, 41, 926956)}
Saving state after processing 790 followers...
Processing follower tomasecu.bsky.social (791/844)...
Fetching profile for tomasecu.bsky.social...
Fetched profile: {'handle': 'tomasecu.bsky.social', 'displayName': 'Tomás', 'description': 'Hello new world. Viñamarino en la capital.', 'followers_count': 83, 'created_at': '2024-11-14T17:28:12.626Z', 'avatar_url': 'https://cdn.bsky.app/img/avatar/plain/did:plc:4gr4hssgan4uy7vzvipwlndb/bafkreiaoz3sx5m5lvajpjg6yitp5kbxvsemqbz2b4l7lmhrd5wjbwdcdsa@jpeg', 'timestamp': datetime.datetime(2024, 12, 4, 20, 39, 42, 249934)}
Processing follower carloalarcon.bsky.social (792/844)...
Fe

Fetched profile: {'handle': 'hwernerf.bsky.social', 'displayName': 'Hans Werner', 'description': 'Chuncho. Ingeniero, peatón y ciclista. Bielsista sampaolista.', 'followers_count': 37, 'created_at': '2024-11-18T11:06:39.546Z', 'avatar_url': 'https://cdn.bsky.app/img/avatar/plain/did:plc:icbm2foren4awny4q53ejr2e/bafkreihc6mepa4xucbhltlphfyl7mlf3u5fwegoro5hi66mzb5jcxzqxty@jpeg', 'timestamp': datetime.datetime(2024, 12, 4, 20, 39, 46, 998605)}
Processing follower alscl.bsky.social (807/844)...
Fetching profile for alscl.bsky.social...
Fetched profile: {'handle': 'alscl.bsky.social', 'displayName': '', 'description': '', 'followers_count': 16, 'created_at': '2024-11-21T03:40:00.770Z', 'avatar_url': 'https://cdn.bsky.app/img/avatar/plain/did:plc:r5bnbazi47rtnuhyd4s7jq3e/bafkreia5u3rqq7g5kwgopbrot3ehcnchkjebqmez2jdr3yqxl3zt467xhq@jpeg', 'timestamp': datetime.datetime(2024, 12, 4, 20, 39, 47, 307984)}
Processing follower danilozurital.bsky.social (808/844)...
Fetching profile for danilozurita

Fetched profile: {'handle': 'ex-tinto.bsky.social', 'displayName': 'ExTinto', 'description': '', 'followers_count': 45, 'created_at': '2024-11-20T16:11:30.771Z', 'avatar_url': 'https://cdn.bsky.app/img/avatar/plain/did:plc:lqfcgggtg6rwqlpzwxq5iiba/bafkreiaqqfcdkcj6ihzeg3vuonsiyfuuoz55qzh5e2xzfb4v2lfgzuwlye@jpeg', 'timestamp': datetime.datetime(2024, 12, 4, 20, 39, 51, 817734)}
Processing follower gaboflowers.bsky.social (822/844)...
Fetching profile for gaboflowers.bsky.social...
Fetched profile: {'handle': 'gaboflowers.bsky.social', 'displayName': 'GаbrieI FM', 'description': 'nunca he aprendido\na tocar mis propias canciones\n(→ músico frustrado)\n\n– accidentally zen\n\nCOMPARTO SOBRE: política, informática (❌Criptoquesitos, ❌DeepLearning), economía, desvaríos personales\n\ninstagram.com/g4boflowers\nMastodon: @gaboflowers@lile.cl', 'followers_count': 47, 'created_at': '2024-11-19T18:00:58.176Z', 'avatar_url': 'https://cdn.bsky.app/img/avatar/plain/did:plc:537q2x47f36pwu2d5l4ula6h/b

Fetched profile: {'handle': 'cwenger.bsky.social', 'displayName': '', 'description': '', 'followers_count': 24, 'created_at': '2024-11-14T14:31:15.226Z', 'avatar_url': 'https://cdn.bsky.app/img/avatar/plain/did:plc:ezk2ukuxi6ax6cidony5hckc/bafkreia2wll7tgtgej5ykrhjapj3srfqn6voxn5yplliixy4vxvvjg7cj4@jpeg', 'timestamp': datetime.datetime(2024, 12, 4, 20, 39, 56, 465355)}
Processing follower nahuel2020.bsky.social (837/844)...
Fetching profile for nahuel2020.bsky.social...
Fetched profile: {'handle': 'nahuel2020.bsky.social', 'displayName': '', 'description': '', 'followers_count': 20, 'created_at': '2024-11-19T11:38:21.605Z', 'avatar_url': 'https://cdn.bsky.app/img/avatar/plain/did:plc:rxdcub3lfwp77fhi7zsa5vkj/bafkreifei6osqmy77uq4f6w6wba24qhspmxpf7lvj2saitlvez3hzu75py@jpeg', 'timestamp': datetime.datetime(2024, 12, 4, 20, 39, 56, 769289)}
Processing follower uber47.bsky.social (838/844)...
Fetching profile for uber47.bsky.social...
Fetched profile: {'handle': 'uber47.bsky.social', 'disp